In [1]:
# Uniform maximum confusion probability density function with straight line.
# Prior belief
# Posterior belief updated by measurement
# Updating beliefs = convolution

import numpy as np

### [X1, X2, X3, X4, X5] = 1.0

### p(Xi) = 0.2 for i = 1...5

In [2]:
p = [0.2, 0.2, 0.2, 0.2, 0.2]

print(p)

[0.2, 0.2, 0.2, 0.2, 0.2]


In [3]:
def create_p_vector(n):
    """Create uniform probability vector with n elements"""
    p = []
    for i in range(n):
        p.append(1./n)
    return np.array(p)

In [4]:
p = create_p_vector(5)
print(p)

[ 0.2  0.2  0.2  0.2  0.2]


In [5]:
measurement = np.array([0.2, 0.6, 0.6, 0.2, 0.2])

In [6]:
posterior = p * measurement

In [7]:
posterior.sum()

0.3600000000000001

In [8]:
# Normalizing to return probability distribution equaling 1
# p(Xi|Z)
# posterior distribution of place Xi given measurement Z
posterior = posterior / posterior.sum()
posterior

array([ 0.11111111,  0.33333333,  0.33333333,  0.11111111,  0.11111111])

In [9]:
posterior.sum()

0.99999999999999978

In [10]:
p = create_p_vector(5)
p

array([ 0.2,  0.2,  0.2,  0.2,  0.2])

In [11]:
# Write code that outputs p after multiplying each entry by pHit or pMiss
# at the appropriate places
# Red cells 1/2 are hits and other green cells are misses
pHit = 0.6
pMiss = 0.2

p[0] = p[0] * pMiss
p[1] = p[1] * pHit
p[2] = p[2] * pHit
p[3] = p[3] * pMiss
p[4] = p[4] * pMiss

In [12]:
print(p.sum())

0.36


In [13]:
p = create_p_vector(5)
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2


### MEASUREMENT UPDATE
def sense(p, Z):
    q = []
    # iterate over all elements in input probability p
    for i in range(len(p)):
        # binary flag if true = 1, if false = 0
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1- hit) * pMiss))
    return q / sum(q)

In [15]:
print(sense(p, measurements))

[ 0.2  0.2  0.2  0.2  0.2]


In [16]:
for k in range(len(measurements)):
    p = sense(p, measurements[k])

In [17]:
p

array([ 0.2,  0.2,  0.2,  0.2,  0.2])

In [18]:
# input distribution p and motion number U (grid cells moving),
# return new distribution q, after the move, if U = 0 q = p
p = np.array([0, 1, 0, 0, 0])
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2

def move(p, U):
    q = []
    for i in range(len(p)):
        q.append(p[(i - U) % len(p)])
    return q

In [19]:
def move(p, U):
    q = []
    for i in range(len(p)):
        q.append(p[i - U % len(p)])
    return q

In [20]:
print(move(p, 1))

[0, 0, 1, 0, 0]


### Inaccurate robot motion
U = 1 

[0., 0.1, 0.8, 0.1, 0.]

U = 2

[0., 0., 0.1, 0.8, 0.1]

========================

U1 = [0., 1., 0., 0., 0.]

p(Xi+2|Xi) = 0.8

p(Xi+1|Xi) = 0.1

p(Xi+3|Xi) = 0.1

U2 = [0., 0., 0.1, 0.8, 0.1]

========================

U1 = [0., 0.5, 0., 0.5, 0]

p(Xi+2|Xi) = 0.8

p(Xi+1|Xi) = 0.1

p(Xi+3|Xi) = 0.1

=========================

In [22]:
U21 = np.array([0., 0., 0.05, 0.4, 0.05])
U22 = np.array([0.4, 0.05, 0., 0., 0.05])
U2 = U21 + U22
U2

array([ 0.4 ,  0.05,  0.05,  0.4 ,  0.1 ])

In [23]:
U1 = create_p_vector(5)

In [24]:
U1

array([ 0.2,  0.2,  0.2,  0.2,  0.2])

p(Xi+2|Xi) = 0.8

p(Xi+1|Xi) = 0.1

p(Xi+3|Xi) = 0.1

In [25]:
# Each grid cell equally likely with motion model
# applied still yields grid cells with equal likelihood.
U2 = U1
U2

array([ 0.2,  0.2,  0.2,  0.2,  0.2])

Modify the move function to accomodate the added probabilities of overshooting or undershooting the intended destination

    def move(p, U):

        q = []
        for i in range(len(p)):
            q.append(p[(i - U) % len(p)])
        return q

In [56]:
p = [0, 1, 0, 0, 0]
world = ['green', 'red', 'red', 'green', 'green']
meansurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

In [57]:
def sense(p, Z):
    q = []
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

In [58]:
def move(p, U):
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1)%len(p)]
        s = s + pUndershoot * p[(i-U+1)%len(p)]
        q.append(s)
    return q

In [59]:
for i in range(1000):
    p = move(p, 1)

In [60]:
print(p)

[0.20000000000000365, 0.20000000000000373, 0.20000000000000365, 0.2000000000000035, 0.2000000000000035]


Measure of information of a distribution

    Entropy
    
    -E p(Xi)log(p(Xi))

Given the list motions=[1,1] which means the robot moves right and then right again, compute the posterior 
distribution if the robot first senses red, then moves 
right one, then senses green, then moves right again, 
starting with a uniform prior distribution.

In [61]:
p = create_p_vector(5)
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1, 1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    q = []
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q

For the number of measurements, compute a sense step and compute a motion step update on p.

In [62]:
for k in range(len(measurements)):
    p = sense(p, measurements[k])
    p = move(p, motions[k])

In [63]:
print(p)

[0.21157894736842103, 0.15157894736842109, 0.08105263157894739, 0.16842105263157897, 0.38736842105263158]


In [65]:
p = create_p_vector(5)
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'red']
motions = [1, 1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1
for k in range(len(measurements)):
    p = sense(p, measurements[k])
    p = move(p, motions[k])
print(p)

[0.078823529411764709, 0.075294117647058845, 0.22470588235294123, 0.43294117647058822, 0.18823529411764706]


## Localization

BELIEF = PROBABILITY

SENSE = PRODUCT FOLLOWED BY NORMALIZATION

MOVE = CONVOLUTION (=ADDITION)

## Formal Definition

    0 <= P(X) <= 1
    
    P(X1) = 0.2
    P(X2) = 0.8
    
    =============
    
    P(X1) = 0
    P(X2) = 1

    =============
    
    0 <= P(X) <= 1
    
    E P(Xi) = 1
    
    [0.1, 0.1, 0.1, 0.1, 0.6]
    
    =============

## Measurements

## BAYES RULE:

    E = sum of
    X = grid cell
    Z = measurement
    P(X) = prior distribution
    P(Z|X) = measurement probability, ie(chances of seeing red/green tile for every possible location)   
    P(Z|Xi) * P(Xi) = non-normalized posterior distribution
    
    p(X|Z) = ( P(Z|Xi) * P(Xi) ) / P(Z)
    p(Z) = E(i) P(Z|Xi) * P(Xi)
    
    Function that assigns to each grid cell a number and p(Z) remains the same.
    
    Non-normalized probability:
    
    Pbar(Xi|Z) = P(Z|Xi) * P(Xi)
    
    alpha(normalizer) = E Pbar(Xi|Z)
    
    Resulting probability:
    
    P(Xi|Z) = ( 1 / alpha ) * Pbar(Xi|Z)
    
    

## CANCER TEST

### BAYES RULE:

#### P(A|B) = P(B|A) * P(A) / P(B)
    
    Cancer/non-cancer = robot position
    
    Pos = color door observed correct

    p(C) = 0.001

    p(nC) = 0.999

    p(POS|C) = 0.8

    p(POS|nC) = 0.1

#### Calculate 2 non-normalized probabilities, calculate alpha and normalize

    pbar(C|POS) = 0.001 * 0.8 = 0.0008
    
    pbar(nC|POS) = 0.999 * 0.1 = 0.0999
    
    alpha = 0.0008 + 0.0999 = 0.1007
    
    p(C|POS) = 0.0008 / 0.1007 = 0.0079
    

### THEOREM OF TOTAL PROBABILITY (convolution)
    
    P(Xi)t = E(j) P(Xj)t-1 * P(Xi|Xj)
    
    P(A) = E(B) P(A|B) * P(B)

### COIN ~ T, H
### P(T) = P(H) = 1/2

    T -> accept
    H -> flip again, accept
    
    p(H2) = ??

### Probability of heads in step 2 =

### Probability of head in step two is =
       
           probability of heads in step two conditional
           on heads in step one times probability of
           heads in one plus probability of heads in step
           two given a tail in step one times the
           probability of tails in step one

    p(H2) = p(H2|H1) * P(H1) + P(H2|T1) * P(T1)
    
          = 1 / 4

### Fair coin p(H) = 0.5
### Loaded coin p(H) = 0.1

#### Grab with 50% change, flip it, observe H
#### p(fair) = ???

    pbar(F|H) = p(H|F) * P(F) = 0.5 * 0.5
    
    pbar(nF|H) = p(H|nF) * p(nF) = 0.1 * 0.5
    
    alpha = 0.25 + 0.05 = 0.3
    
    p(fair) = normalize pbar(F|H)
    
    p(fair) = 0.25 / 0.3 = 0.833

### Problem set 1

#### Question 1

#### If probability of X = 0.2 then probability of not X = 0.8
    P(X) = 0.2
    P(nX) = 0.8
    
#### If probability of X = 0.2 and probability of Y = 0.2 and if X/Y are independent events the probability of X and Y, P(X, Y) is 0.02 * 0.02 = 0.04 (independent probabilities are multiplied)
   
    P(X) = 0.2
    P(Y) = 0.2 [X/Y independent]
    P(X, Y) = 0.2 * 0.2 = 0.04
    
#### If probability of X = 0.2, and probability of Y given X = 0.6, and probability of Y given not X = 0.6, the probability of Y is 0.6, because not matter value of X, Y has the same probability
    P(X) = 0.2
    P(nX) = 0.8
    P(Y|X) = 0.6
    P(Y|nX) = 0.6
    P(Y) = 0.6
    
    Bayes rule proof:  p(Z) = E(i) P(Z|Xi) * P(Xi)
    P(Y) = P(Y|X) * P(X) + P(Y|nX) * P(nX)
    P(Y) = 0.6 * 0.2 + 0.6 * 0.8 = 0.6

#### Robot in 2D has an x coordinate, y coordinate and theta (heading)

#### Robot in 3D has an x coordinate, y coordinate, z coordinate and 3 rotational angles (roll, pitch and yaw)

#### Increase in dimensions results in exponential increase in number of variables

### Quenstion 2

### Bayes rule:

    P(A|B) = P(B|A) * P(A) / P(B)
    
    P(F) = 0.001
    P(nF) = 0.999
    B = Neighbour says "it burns"
    P(B) = 0.9 "Burning"
    P(nB) = 0.1 "Not burning"
    
#### Probability of neighbour saying there is a fire and there actually being a fire times the probability of a fire occuring
    pbar(F|B) = p(B|F) * P(F)
    pbar(F|B) = 0.9 * 0.001 = 0.0009

#### Probability of neighbour saying there is a fire and there not being a fire times the probability of a fire not occuring
    pbar(nF|B) = p(B|nF) * p(nF)
    pbar(nF|B) = 0.1 * 0.999 = 0.0999
    
#### Normalizer
    1 / (sum of non-normalized posterior probabilities)
    
    1 / (0.0009 + 0.0999) = 9.9206
    
#### Normalized posterior probabilities (note that multipling by 1/normalizer is the same as simply dividing by normalizer

#### Probability of neighbour saying there is a fire and there actually being a fire
    
    P(F|B) = 0.0009 * 9.9206 =   0.00892
    
#### Probability of neighbour saying there is a fire and there not being a fire
    
    P(nF|B) = 0.0999 * 9.9206 = 0.99106

### Localization program (hard)

    

In [102]:
# nice solution from:
# https://github.com/jeremy-shannon/udacity-AI-for-robotics/blob/master/1%20-%20Localization/localization.py

In [103]:
sensor_right = 0.7
sensor_wrong = 1.0 - sensor_right
p_move = 0.8

# Motion failure
p_stay = 1.0 - p_move

colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]

measurements = ['G','G','G','G','G']

motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]

In [104]:
def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
    # >>> Insert your code here <<<
    for i in range(len(measurements)):
        p = move(p, motions[i], p_move, colors)
        p = sense(p, measurements[i], sensor_right, colors)
    return p

In [105]:
def sense(p, Z, pHit, colors):
    pMiss = 1. - pHit
    q = [[0 for row in range(len(colors[0]))] for col in range(len(colors))]
    for x in range(len(p[0])):
        for y in range(len(p)):
            hit = (Z == colors[y][x]) 
            q[y][x] = (p[y][x] * (hit * pHit + (1-hit) * pMiss))
    s = sum(map(sum,q))
    for x in range(len(q[0])):
        for y in range(len(q)):
            q[y][x] = q[y][x] / s
    return q

In [106]:
def move(p, U, p_move, colors):
    p_stay = 1. - p_move
    q = [[0 for row in range(len(colors[0]))] for col in range(len(colors))]
    for x in range(len(q[0])):
        for y in range(len(q)):
            s = p_move * p[(y-U[0]) % len(p)][(x-U[1]) % len(p[0])]
            s += p_stay * p[y][x]
            q[y][x] = s
    return q

In [107]:
def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print('[' + ',\n '.join(rows) + ']')

In [108]:
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)

In [109]:
show(p)

[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]
